In [1]:
# Manipulación de datos
import pandas as pd
import numpy as np

# Utilidades generales
import time
import os
from itertools import combinations

# Visualización
import matplotlib.pyplot as plt
import plotly.express as px
import networkx as nx
import plotly.graph_objects as go

# Algoritmos y matrices
from scipy.sparse import csr_matrix
from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.preprocessing import TransactionEncoder

## Preprocesamiento y limpieza de datasets

In [2]:
#Cargar datases
orders= pd.read_csv('order_products__train.csv')
products = pd.read_csv('products.csv')

# Limpieza de la base de datos
products['product_name'] = products['product_name'].replace(
    to_replace="[^a-zA-Z\\d\\s]+",
    value="~",
    regex=True
).str.lower().str.strip()


In [3]:
# Mege y agrupación por No de  orden
dfMerged = pd.merge(orders, products, on="product_id", how="inner")
dfMerged = dfMerged.sort_values("order_id").reset_index(drop=True)

# Agrupar productos por transacción (cada pedido (order_id) se agrupa en una lista de productos)
transactions = dfMerged.groupby('order_id')['product_name'].apply(list).tolist()

for t in transactions[:5]:
    print(t)

['bulgarian yogurt', 'organic 4~ milk fat whole milk cottage cheese', 'cucumber kirby', 'organic celery hearts', 'lightly smoked sardines in olive oil', 'organic hass avocado', 'bag of organic bananas', 'organic whole string cheese']
['grated pecorino romano cheese', 'spring water', 'super greens salad', 'cage free extra large grade aa eggs', 'prosciutto~ americano', 'organic garnet sweet potato ~yam~', 'asparagus', 'organic half ~ half']
['shelled pistachios', 'organic raw unfiltered apple cider vinegar', 'organic hot house tomato', 'organic baby arugula', 'green peas', 'bunched cilantro', 'flat parsley~ bunch', 'organic biologique limes', 'fresh dill']
['organic raspberries', 'organic whole strawberries', 'organic blueberries', 'organic grape tomatoes', 'organic cucumber', 'roasted turkey', 'organic pomegranate kernels']
['natural spring water', 'organic unsweetened almond milk', 'tomatoes~ crushed~ organic', 'organic sliced provalone cheese', 'organic chocolate almondmilk pudding', 

In [5]:
# Exportando las transacciones a CSV (formato basket) para su uso posterior en R.

output_csv = "TransactionsInstacart.csv"
with open(output_csv, "w", newline="", encoding="utf-8") as f:
    for trans in transactions:
        f.write(",".join(trans) + "\n")
print(f"Transacciones exportadas a '{output_csv}'.")

Transacciones exportadas a 'TransactionsInstacart.csv'.


## Creacion de Martiz dispersa

In [6]:
# Se crea un mapeo de cada producto a un índice.
item_mapping = {item: idx for idx, item in enumerate(sorted({item for transaction in transactions for item in transaction}))}

# Recorremos cada transacción para construir las coordenadas (filas y columnas) en la matriz.
rows, cols = [], []
for row_idx, trans in enumerate(transactions):
    for item in trans:
        rows.append(row_idx)
        cols.append(item_mapping[item])

order_matrix_sparse = csr_matrix(([1] * len(rows), (rows, cols)), shape=(len(transactions), len(item_mapping)))
print(f"Matriz dispersa creada con forma: {order_matrix_sparse.shape}")

Matriz dispersa creada con forma: (131209, 39043)


In [7]:
# Convertir la matriz dispersa a una lista de transacciones (lista de productos comprados)
transactions_list = []
for row in order_matrix_sparse:
    non_zero_indices = row.nonzero()[1]  # Encuentra los índices no cero
    transaction = [list(item_mapping.keys())[index] for index in non_zero_indices]
    transactions_list.append(transaction)

## Implementación Manual del Algoritmo Apriori en Python (Procesamiento por Lotes)

In [9]:
def generar_candidatos(itemsets_frecuentes_k, k, max_candidates=100000, support_dict=None):
    """
    Genera candidatos (k+1)-itemsets a partir de itemsets frecuentes de tamaño k.
    Aplica límites si hay demasiados itemsets y usa un índice por prefijo para mayor eficiencia.
    """
    if len(itemsets_frecuentes_k) > 1000:
        print(f"Demasiados itemsets frecuentes ({len(itemsets_frecuentes_k)}). Limitando a los 1000 más frecuentes.")
        if support_dict:
            itemsets_frecuentes_k = sorted(itemsets_frecuentes_k,
                                           key=lambda x: support_dict.get(tuple(x), 0),
                                           reverse=True)[:1000]
        else:
            itemsets_frecuentes_k = itemsets_frecuentes_k[:1000]

    n = len(itemsets_frecuentes_k)
    estimated_candidates = n * (n - 1) // 2
    if estimated_candidates > max_candidates:
        reduction_factor = max_candidates / estimated_candidates
        limit = int(n * (reduction_factor ** 0.5))
        print(f"Estimación de candidatos ({estimated_candidates}) excede el máximo. Limitando a {limit} itemsets.")
        if support_dict:
            itemsets_frecuentes_k = sorted(itemsets_frecuentes_k,
                                           key=lambda x: support_dict.get(tuple(x), 0),
                                           reverse=True)[:limit]
        else:
            itemsets_frecuentes_k = itemsets_frecuentes_k[:limit]
        n = len(itemsets_frecuentes_k)

    candidatos_set = set()
    if k == 1:
        for i in range(n):
            for j in range(i+1, n):
                candidato = tuple(sorted([itemsets_frecuentes_k[i][0], itemsets_frecuentes_k[j][0]]))
                candidatos_set.add(candidato)
    else:
        # Crear índice por prefijo (los primeros k-1 elementos)
        prefix_index = {}
        for idx, itemset in enumerate(itemsets_frecuentes_k):
            prefix = tuple(itemset[:k-1])
            prefix_index.setdefault(prefix, []).append(idx)
        for prefix, indices in prefix_index.items():
            if len(indices) > 1:
                for i in range(len(indices)):
                    for j in range(i+1, len(indices)):
                        idx1, idx2 = indices[i], indices[j]
                        candidato = list(prefix) + [itemsets_frecuentes_k[idx1][k-1],
                                                    itemsets_frecuentes_k[idx2][k-1]]
                        candidato.sort()
                        candidatos_set.add(tuple(candidato))

    candidatos_unicos = [list(x) for x in candidatos_set]
    if len(candidatos_unicos) > max_candidates:
        print(f"Aún hay demasiados candidatos ({len(candidatos_unicos)}). Limitando a {max_candidates}.")
        candidatos_unicos = candidatos_unicos[:max_candidates]

    return candidatos_unicos

In [10]:
#Optimización en lugar de usar sorted en cada iteración dentro de all , se preordenan los candidatos y se optimiza la comparación
def poda_apriori(candidatos, itemsets_frecuentes_k, k):
    """
    Poda candidatos eliminando aquellos cuyo subconjunto (de tamaño k) no es frecuente.
    """
    itemsets_frecuentes_set = set(tuple(sorted(x)) for x in itemsets_frecuentes_k)  # Preordena los itemsets frecuentes
    candidatos_podados = [candidato for candidato in candidatos if all(tuple(sorted(candidato[:i] + candidato[i+1:])) in itemsets_frecuentes_set for i in range(len(candidato)))]
    return candidatos_podados

In [11]:
def apriori_lotes(transactions_list, min_support, batch_size=50000):
    """
    Implementación manual del algoritmo Apriori usando procesamiento por lotes.
    Calcula 1-itemsets y itemsets de mayor tamaño, usando un diccionario para los soportes.
    """
    n_total = len(transactions_list)
    print(f"Procesando {n_total} transacciones con soporte mínimo {min_support}")

    # Fase 1: Contar 1-itemsets en lotes
    item_counts = {}
    n_batches = (n_total + batch_size - 1) // batch_size
    print(f"Fase 1: Generando 1-itemsets frecuentes en {n_batches} lotes.")
    for i in range(n_batches):
        start_idx = i * batch_size
        end_idx = min((i + 1) * batch_size, n_total)
        print(f"  Procesando lote {i+1}/{n_batches} (transacciones {start_idx} - {end_idx})")
        batch_transactions = transactions_list[start_idx:end_idx]
        for transaction in batch_transactions:
            for item in transaction:
                item_counts[item] = item_counts.get(item, 0) + 1

    frequent_1_itemsets = []
    support_dict = {}
    for item, count in item_counts.items():
        support = count / n_total
        if support >= min_support:
            frequent_1_itemsets.append([item])
            support_dict[tuple([item])] = support
    print(f"Se encontraron {len(frequent_1_itemsets)} 1-itemsets frecuentes.")

    all_frequent_itemsets = {1: frequent_1_itemsets}
    k = 1
    while all_frequent_itemsets.get(k, []):
        print(f"Fase {k+1}: Generando {k+1}-itemsets frecuentes.")
        if len(all_frequent_itemsets[k]) > 1000:
            print(f"  Demasiados itemsets frecuentes ({len(all_frequent_itemsets[k])}). Limitando a 1000 con mayor soporte.")
            sorted_itemsets = sorted(all_frequent_itemsets[k],
                                     key=lambda x: support_dict.get(tuple(x), 0),
                                     reverse=True)[:1000]
            all_frequent_itemsets[k] = sorted_itemsets

        candidatos = generar_candidatos(all_frequent_itemsets[k], k, support_dict=support_dict)
        if len(candidatos) > 100000:
            print(f"  Demasiados candidatos ({len(candidatos)}). Aumentando temporalmente el umbral de soporte.")
            temp_min_support = min_support * 2
            filtered_itemsets = [itemset for itemset in all_frequent_itemsets[k] if support_dict.get(tuple(itemset), 0) >= temp_min_support]
            candidatos = generar_candidatos(filtered_itemsets, k, support_dict=support_dict)

        candidatos_podados = poda_apriori(candidatos, all_frequent_itemsets[k], k)
        print(f"  Generados {len(candidatos)} candidatos, {len(candidatos_podados)} después de poda.")
        if not candidatos_podados:
            break

        # Contar soporte de candidatos en lotes
        candidato_counts = {tuple(c): 0 for c in candidatos_podados}
        for i in range(n_batches):
            start_idx = i * batch_size
            end_idx = min((i + 1) * batch_size, n_total)
            print(f"  Contando candidatos en lote {i+1}/{n_batches} (transacciones {start_idx} - {end_idx})")
            batch_transactions = transactions_list[start_idx:end_idx]
            for transaction in batch_transactions:
                transaction_set = set(transaction)
                for candidato in candidatos_podados:
                    if all(item in transaction_set for item in candidato):
                        candidato_counts[tuple(candidato)] += 1

        frequent_itemsets_k_plus_1 = []
        for candidato, count in candidato_counts.items():
            support = count / n_total
            if support >= min_support:
                frequent_itemsets_k_plus_1.append(list(candidato))
                support_dict[tuple(sorted(candidato))] = support

        print(f"  Se encontraron {len(frequent_itemsets_k_plus_1)} {k+1}-itemsets frecuentes.")
        if frequent_itemsets_k_plus_1:
            all_frequent_itemsets[k+1] = frequent_itemsets_k_plus_1
            k += 1
        else:
            break
    return all_frequent_itemsets, support_dict

In [12]:
def generar_reglas(itemsets_frecuentes, support_dict, min_confidence=0.3):
    """
    Genera reglas de asociación a partir de itemsets frecuentes.
    Para cada itemset (de tamaño >= 2), genera todas las posibles divisiones:
    antecedente y consecuente, calcula métricas (soporte, confianza, lift) y
    retorna un DataFrame con las reglas ordenadas por lift.
    """
    rules = []
    for k in range(2, len(itemsets_frecuentes) + 1):
        if k not in itemsets_frecuentes:
            continue
        for itemset in itemsets_frecuentes[k]:
            itemset_support = support_dict.get(tuple(sorted(itemset)), 0)
            for i in range(1, len(itemset)):
                for antecedente_items in combinations(itemset, i):
                    antecedente = list(antecedente_items)
                    consecuente = [item for item in itemset if item not in antecedente]
                    if not antecedente or not consecuente:
                        continue
                    antecedente_support = support_dict.get(tuple(sorted(antecedente)), 0)
                    if antecedente_support == 0:
                        continue
                    confidence = itemset_support / antecedente_support
                    if confidence >= min_confidence:
                        consecuente_support = support_dict.get(tuple(sorted(consecuente)), 0)
                        if consecuente_support == 0:
                            continue
                        lift = confidence / consecuente_support
                        rules.append({
                            'antecedent': antecedente,
                            'consequent': consecuente,
                            'support': itemset_support,
                            'confidence': confidence,
                            'lift': lift
                        })
    if rules:
        rulesDf = pd.DataFrame(rules)
        rulesDf = rulesDf.sort_values('lift', ascending=False).reset_index(drop=True)
        return rulesDf
    else:
        return pd.DataFrame(columns=['antecedent', 'consequent', 'support', 'confidence', 'lift'])

## Ejecución de Apriori Manual

In [13]:
# Ejecutar el algoritmo Apriori manual
min_support = 0.01
min_confidence = 0.3

# Aquí usamos la lista de transacciones obtenida antes
transactions_list = transactions

start_time = time.time()

all_frequent_itemsets, support_dict = apriori_lotes(transactions_list, min_support)

# Generar reglas a partir de los itemsets frecuentes
rules_manual = generar_reglas(all_frequent_itemsets, support_dict, min_confidence=0.3)
print("\nReglas generadas manualmente (primeras 5):")
print(rules_manual.head())


manual_time = time.time() - start_time
print(f"\nTiempo total del Apriori manual (itemsets + reglas): {manual_time:.2f} segundos")


print("\nReglas generadas manualmente (primeras 5):")
print(rules_manual.head())


Procesando 131209 transacciones con soporte mínimo 0.01
Fase 1: Generando 1-itemsets frecuentes en 3 lotes.
  Procesando lote 1/3 (transacciones 0 - 50000)
  Procesando lote 2/3 (transacciones 50000 - 100000)
  Procesando lote 3/3 (transacciones 100000 - 131209)
Se encontraron 104 1-itemsets frecuentes.
Fase 2: Generando 2-itemsets frecuentes.
  Generados 5356 candidatos, 5356 después de poda.
  Contando candidatos en lote 1/3 (transacciones 0 - 50000)
  Contando candidatos en lote 2/3 (transacciones 50000 - 100000)
  Contando candidatos en lote 3/3 (transacciones 100000 - 131209)
  Se encontraron 16 2-itemsets frecuentes.
Fase 3: Generando 3-itemsets frecuentes.
  Generados 22 candidatos, 7 después de poda.
  Contando candidatos en lote 1/3 (transacciones 0 - 50000)
  Contando candidatos en lote 2/3 (transacciones 50000 - 100000)
  Contando candidatos en lote 3/3 (transacciones 100000 - 131209)
  Se encontraron 0 3-itemsets frecuentes.

Reglas generadas manualmente (primeras 5):
     

## Ejecución de Apriori con mlxtend

In [14]:
# Convertir la lista de transacciones en formato binarizado para la librería
te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)
df = pd.DataFrame(te_ary, columns=te.columns_)

# Cronometrar todo el flujo de Apriori con mlxtend
start_time_mlxtend = time.time()

# Generar itemsets frecuentes
frequent_itemsets = apriori(df, min_support=min_support, use_colnames=True)

# Generar reglas de asociación
rules_mlxtend = association_rules(frequent_itemsets, metric="lift", min_threshold=1)

# Detener el cronómetro
mlxtend_time = time.time() - start_time_mlxtend
print(f"\nTiempo total del Apriori con mlxtend (itemsets + reglas): {mlxtend_time:.2f} segundos")

# Mostrar resultados
print("\nItemsets frecuentes generados con mlxtend:")
print(frequent_itemsets.head())
print("\nReglas generadas con mlxtend (primeras 5):")
print(rules_mlxtend.head())



Tiempo total del Apriori con mlxtend (itemsets + reglas): 83.29 segundos

Itemsets frecuentes generados con mlxtend:
    support                    itemsets
0  0.017514    (100~ whole wheat bread)
1  0.011737       (2~ reduced fat milk)
2  0.017163  (apple honeycrisp organic)
3  0.029480                 (asparagus)
4  0.117980    (bag of organic bananas)

Reglas generadas con mlxtend (primeras 5):
                antecedents               consequents  antecedent support  \
0  (bag of organic bananas)    (organic baby spinach)            0.117980   
1    (organic baby spinach)  (bag of organic bananas)            0.074568   
2  (bag of organic bananas)    (organic hass avocado)            0.117980   
3    (organic hass avocado)  (bag of organic bananas)            0.055583   
4  (bag of organic bananas)     (organic raspberries)            0.117980   

   consequent support   support  confidence      lift  representativity  \
0            0.074568  0.017042    0.144444  1.937082       

## Entorno Híbrido con R

In [8]:
# En este oasi ejecutal el pip install rpy2 en la terminal, luego ejecuta el codigo. Esto se debe aque si lo trabjamos en un IDE local este paso se procede en la terminal
import rpy2.robjects as robjects
from rpy2.robjects.packages import importr

ModuleNotFoundError: No module named 'rpy2'

In [ ]:
# Cargar la extensión de R para poder ejecutar código R en celdas de Jupyter
%load_ext rpy2.ipython

# Esta celda ejecutará código R (todo lo que esté después de esta línea será interpretado como código R)
%%R
install.packages("arules")

In [ ]:
%%R -o r_execution_time,rules_df
# Cargar el paquete arules
if (!require("arules")) {
    install.packages("arules", repos="http://cran.us.r-project.org")
    library(arules)
}

# Leer las transacciones desde el archivo exportado por Python
baskets <- read.transactions("TransactionsInstacart.csv", sep = ",", format = "basket")

# Visualizar la frecuencia de ítems
itemFrequencyPlot(baskets, support = 0.055, main = "Frecuencia de ítems (>= 5.5% soporte)")
start_r_time <- Sys.time()

# Generar reglas de asociación con Apriori
rules_r <- apriori(baskets, parameter = list(support = 0.001, confidence = 0.3, minlen = 2, maxlen = 15))
rules_r <- sort(rules_r, by = "lift")

# Detener el cronómetro
end_r_time <- Sys.time()
r_execution_time <- end_r_time - start_r_time
print(paste("Tiempo total de procesamiento en R:", r_execution_time, "segundos"))

# Inspeccionar las primeras reglas
inspect(rules_r[1:5])

# Convertir las reglas en un data.frame y añadir columnas
rules_df <- as(rules_r, "data.frame")
rules_df$antecedent <- labels(lhs(rules_r))  # Extrae el lado izquierdo (lhs)
rules_df$consequent <- labels(rhs(rules_r))  # Extrae el lado derecho (rhs)

# Exportar a un archivo CSV
write.csv(rules_df, file = "rulesdatabase_r.csv", row.names = FALSE, quote = TRUE)


In [ ]:
import os #se utiliza aquí para verificar si un archivo (rulesdatabase_r.csv) existe en el sistema de archivos.


# Verifica si el archivo generado por R está disponible
if os.path.exists("rulesdatabase_r.csv"):
    # Cargar las reglas generadas por R/arules
    rules_r_df = pd.read_csv("rulesdatabase_r.csv")
    rules_r_df.sort_values(by="lift", ascending=False, inplace=True)
    rules_r_df.reset_index(drop=True, inplace=True)
    print("\nReglas generadas por R (primeras 5):")
    print(rules_r_df.head())
else:
    print("El archivo 'rulesdatabase_r.csv' no se encontró. Asegúrate de haber generado las reglas en R.")


In [ ]:
# Cargar las reglas generadas desde R
rules_r = pd.read_csv("rulesdatabase_r.csv")

## Comparación de las reglas generadas

In [ ]:
def comparar_reglas(rules_py, rules_ml, rules_r):
    """
    Compara reglas generadas manualmente, con mlxtend, y en R.
    """
    # Normalizar `rules_mlxtend`: convertir `antecedents` y `consequents` de conjuntos a listas ordenadas
    rules_ml['antecedents'] = rules_ml['antecedents'].apply(lambda x: sorted(list(x)) if isinstance(x, set) else x)
    rules_ml['consequents'] = rules_ml['consequents'].apply(lambda x: sorted(list(x)) if isinstance(x, set) else x)

    # `rules_r` ya tiene las columnas correctas: `antecedent` y `consequent`
    matches = []
    differences = []

    for _, rule_py in rules_py.iterrows():
        # Buscar regla equivalente en `mlxtend`
        matched_ml = rules_ml[
            (rules_ml['antecedents'] == sorted(rule_py['antecedent'])) &
            (rules_ml['consequents'] == sorted(rule_py['consequent']))
        ]

        # Buscar regla equivalente en `R`
        matched_r = rules_r[
            (rules_r['antecedent'] == sorted(rule_py['antecedent'])) &
            (rules_r['consequent'] == sorted(rule_py['consequent']))
        ]

        if not matched_ml.empty and not matched_r.empty:
            matched_ml_row = matched_ml.iloc[0]
            matched_r_row = matched_r.iloc[0]
            matches.append({
                'antecedent': rule_py['antecedent'],
                'consequent': rule_py['consequent'],
                'support_manual': rule_py['support'],
                'support_ml': matched_ml_row['support'],
                'support_r': matched_r_row['support'],
                'confidence_manual': rule_py['confidence'],
                'confidence_ml': matched_ml_row['confidence'],
                'confidence_r': matched_r_row['confidence'],
                'lift_manual': rule_py['lift'],
                'lift_ml': matched_ml_row['lift'],
                'lift_r': matched_r_row['lift']
            })
        else:
           # Si la regla no está en una o más implementaciones
            differences.append({
                'antecedent': rule_py['antecedent'],
                'consequent': rule_py['consequent'],
                'reason': 'No encontrada en mlxtend o R'
            })

    return pd.DataFrame(matches), pd.DataFrame(differences)


In [ ]:
# Recopilar datos


resultados = [
    {
        "Reglas generadas": len(rules_manual),  # Número de reglas generadas manualmente
        "Soporte mínimo": "1%",
        "Confianza promedio": round(rules_manual["confidence"].mean(), 2),  # Promedio de confianza en reglas manuales
        "Lift promedio": round(rules_manual["lift"].mean(), 2),  # Promedio de lift en reglas manuales
        "Tiempo (Python manual)": round(manual_time, 2),  # Tiempo total calculado para la implementación manual
        "Tiempo (`mlxtend`)": round(mlxtend_time, 2),    # Tiempo total calculado con mlxtend
        "Tiempo (R)": round(float(r_execution_time[0]), 2),  # Tiempo calculado en R (convertido a segundos)



    },
]

# Crear un DataFrame con los resultados
df_resultados = pd.DataFrame(resultados)

# Visualizar la tabla
print("\nTabla comparativa de tiempos y resultados:")
from IPython.display import display
display(df_resultados)

In [ ]:
# Datos para el gráfico
soportes = df_resultados["Soporte mínimo"]
tiempos_python = df_resultados["Tiempo (Python manual)"]
tiempos_mlxtend = df_resultados["Tiempo (`mlxtend`)"]
tiempos_r = df_resultados["Tiempo (R)"]
confianza_prom = df_resultados["Confianza promedio"]
lift_prom = df_resultados["Lift promedio"]
reglas_generadas = df_resultados["Reglas generadas"]

# Gráfico 1: Comparación de tiempos
plt.figure(figsize=(10, 6))
plt.plot(soportes, tiempos_python, label="Python manual", marker="o")
plt.plot(soportes, tiempos_mlxtend, label="`mlxtend`", marker="o")
plt.plot(soportes, tiempos_r, label="R", marker="o")
plt.title("Comparación de Tiempos de Ejecución", fontsize=14)
plt.xlabel("Soporte mínimo", fontsize=12)
plt.ylabel("Tiempo (segundos)", fontsize=12)
plt.legend(fontsize=12)
plt.grid(True)
plt.show()

## Generación de Recomendaciones Basadas en una Transacción Existente

In [ ]:
print(rules_df.columns)
print(rules_df.head())


In [ ]:
# Función para convertir una cadena del antecedente a lista (si es necesario)
def convertir_a_lista(cadena):
    cadena = cadena.strip("{}")
    if not cadena:
        return []
    return [item.strip() for item in cadena.split(',')]

# Función para evaluar coincidencia parcial
def coincide_parcial(antecedente, transaccion, umbral=0.5):
    # Retorna True si al menos "umbral" (por defecto 50%) de los ítems del antecedente están en la transacción
    if not antecedente:
        return False
    interseccion = set(antecedente).intersection(set(transaccion))
    return len(interseccion) / len(antecedente) >= umbral

In [ ]:
# Seleccionar una transacción existente por indice
indice_transaccion = 301  # Modifica este valor para probar con distintos índices
transaccion_existente = transactions[indice_transaccion]
print(f"Usando la transacción existente en el índice {indice_transaccion}: {transaccion_existente}")

# Filtrar reglas (obtenidas desde R y almacenadas en 'rules_df') usando el criterio de coincidencia parcial
productos_recomendados = []
for _, regla in rules_df.iterrows():
    # Convertir el antecedente (si viene como cadena) a una lista de ítems
    antecedente = convertir_a_lista(regla['antecedent'])
    # Se evalúa la coincidencia parcial: al menos el 50% de los ítems del antecedente deben estar en la transacción
    if coincide_parcial(antecedente, transaccion_existente, umbral=0.5):
        productos_recomendados.append({
            'producto': regla['consequent'],
            'lift': regla['lift'],
            'confidence': regla['confidence'],
            'support': regla['support']
        })

# Convertir la lista de recomendaciones a un DataFrame definiendo las columnas esperadas
import pandas as pd
columnas = ['producto', 'lift', 'confidence', 'support']
recomendaciones_df = pd.DataFrame(productos_recomendados, columns=columnas)

# Verificar si se encontraron recomendaciones
if recomendaciones_df.empty:
    print("No se encontraron recomendaciones para esta transacción.")
else:
    # Eliminar duplicados y ordenar las recomendaciones por 'lift'
    recomendaciones_df = recomendaciones_df.drop_duplicates(subset='producto').sort_values(by='lift', ascending=False)
    print("Productos recomendados basados en la transacción existente:")
    print(recomendaciones_df.head())

In [ ]:
# Visualización interactiva con Plotly
import plotly.express as px
fig = px.scatter(
    recomendaciones_df,
    x='support',
    y='confidence',
    size='lift',
    color='lift',
    hover_data=['producto'],
    title=f"Productos Recomendados Basados en Transacción Existente (Índice {indice_transaccion})"
)
fig.update_layout(
    xaxis_title="Soporte",
    yaxis_title="Confianza",
    legend_title="Lift",
    template="plotly_dark"
)
fig.show()

# Grafo interactivo con la transacción existente
import networkx as nx
import plotly.graph_objects as go

# Crear grafo para la transacción existente
G = nx.Graph()

# Añadir nodos para la transacción inicial
for producto in transaccion_existente:
    G.add_node(producto, type='comprado', color='blue', size=30)

# Añadir nodos para las recomendaciones
for _, row in recomendaciones_df.iterrows():
    G.add_node(row['producto'], type='recomendado', color='orange', size=row['lift']*10)
    for producto in transaccion_existente:
        G.add_edge(producto, row['producto'], weight=row['lift'])

# Extraer posiciones para los nodos
pos = nx.spring_layout(G)

# Nodos del grafo
node_x = []
node_y = []
node_size = []
node_color = []
node_text = []
for node in G.nodes(data=True):
    node_x.append(pos[node[0]][0])
    node_y.append(pos[node[0]][1])
    node_size.append(node[1]['size'])
    node_color.append(node[1]['color'])
    node_text.append(node[0])

node_trace = go.Scatter(
    x=node_x,
    y=node_y,
    mode='markers',
    marker=dict(size=node_size, color=node_color, line=dict(width=1)),
    text=node_text,
    hoverinfo='text'
)

# Conexiones del grafo
edge_x = []
edge_y = []
for edge in G.edges(data=True):
    edge_x.extend([pos[edge[0]][0], pos[edge[1]][0], None])
    edge_y.extend([pos[edge[0]][1], pos[edge[1]][1], None])

edge_trace = go.Scatter(
    x=edge_x,
    y=edge_y,
    line=dict(width=0.5, color='gray'),
    hoverinfo='none',
    mode='lines'
)

# Visualización interactiva del grafo
fig = go.Figure(data=[edge_trace, node_trace],
                layout=go.Layout(
                    title=f"Red Interactiva de Productos Comprados y Recomendados (Transacción Existente, Índice {indice_transaccion})",
                    showlegend=False,
                    xaxis=dict(showgrid=False, zeroline=False),
                    yaxis=dict(showgrid=False, zeroline=False)
                ))
fig.show()

## Comparativas de rendimiento

In [15]:
#Esta funcion es para generar la documentacion de complejidad
def generar_dataset_sintetico(n_transacciones, n_items, longitud_media):

    items = [f'item_{i}' for i in range(n_items)]
    transactions = []

    for _ in range(n_transacciones):
        # Determinar longitud de esta transacción
        longitud = max(1, int(np.random.normal(longitud_media, longitud_media/4)))
        longitud = min(longitud, n_items)  # No puede ser mayor que el número total de items

        # Generar transacción
        transaction = random.sample(items, longitud)
        transactions.append(transaction)

    return transactions

In [16]:
def contar_items_individuales(transactions):

    item_counts = {}
    for transaction in transactions:
        for item in transaction:
            if item not in item_counts:
                item_counts[item] = 0
            item_counts[item] += 1
    return item_counts

In [17]:
#Esta funcion es para generar la documentacion de complejidad
def generar_dataset_sintetico(n_transacciones, n_items, longitud_media):

    items = [f'item_{i}' for i in range(n_items)]
    transactions = []

    for _ in range(n_transacciones):
        # Determinar longitud de esta transacción
        longitud = max(1, int(np.random.normal(longitud_media, longitud_media/4)))
        longitud = min(longitud, n_items)  # No puede ser mayor que el número total de items

        # Generar transacción
        transaction = random.sample(items, longitud)
        transactions.append(transaction)

    return transactions

In [18]:
def apriori_original(transactions, min_support=0.5, max_length=None):

    # Convertimos transacciones a conjuntos si no lo son ya
    transactions = [set(transaction) for transaction in transactions]
    n_transactions = len(transactions)


    if isinstance(min_support, list) or isinstance(min_support, tuple):
        min_support = float(min_support[0]) if min_support else 0.5

    # Calcular el conteo mínimo como un número entero
    min_support_count = int(min_support * n_transactions)

    # Imprimir para depuración
    # print(f"DEBUG - min_support: {min_support} (tipo: {type(min_support)})")
    # print(f"DEBUG - min_support_count: {min_support_count} (tipo: {type(min_support_count)})")

    # Paso 1: Generar 1-itemsets frecuentes
    item_counts = {}
    for transaction in transactions:
        for item in transaction:
            if frozenset([item]) not in item_counts:
                item_counts[frozenset([item])] = 0
            item_counts[frozenset([item])] += 1

    # Filtrar por soporte mínimo - usando un bucle explícito para evitar problemas
    L1 = {}
    for itemset, count in item_counts.items():
        # Verificar que count sea un número
        if isinstance(count, (int, float)) and isinstance(min_support_count, (int, float)):
            if count >= min_support_count:
                L1[itemset] = count

    # Almacenar todos los itemsets frecuentes
    all_frequent_itemsets = dict(L1)
    current_L = L1
    k = 2

    # Iteración principal
    while current_L and (max_length is None or k <= max_length):
        # Paso 2: Generar candidatos de tamaño k
        Ck = {}
        for i in current_L:
            for j in current_L:
                # Unir itemsets que comparten k-2 elementos
                union = i.union(j)
                if len(union) == k:
                    # Verificar si todos los subconjuntos son frecuentes
                    all_subsets_frequent = True
                    for item in union:
                        subset = union - frozenset([item])
                        if subset not in current_L:
                            all_subsets_frequent = False
                            break

                    if all_subsets_frequent:
                        Ck[union] = 0

        # Paso 3: Contar soporte para candidatos
        for transaction in transactions:
            for candidate in Ck:
                if candidate.issubset(transaction):
                    Ck[candidate] += 1

        # Filtrar por soporte mínimo
        current_L = {itemset: count for itemset, count in Ck.items()
                    if count >= min_support_count}

        # Actualizar resultados
        all_frequent_itemsets.update(current_L)
        k += 1

    return all_frequent_itemsets

In [ ]:
def comparar_rendimiento(datasets, min_supports, implementaciones, parametros=None):
    if parametros is None:
        # Por defecto, todas las implementaciones usan 'min_support'
        parametros = {nombre: 'min_support' for nombre in implementaciones}

    resultados = defaultdict(list)

    for dataset_name, dataset in datasets.items():
        print(f"\nEvaluando dataset: {dataset_name}")

        for min_support in min_supports:
            print(f"\n  Soporte mínimo: {min_support}")

            for nombre_impl, funcion_impl in implementaciones.items():
                print(f"    Ejecutando {nombre_impl}...", end=" ")

                # Medir tiempo de ejecución
                inicio = time.time()

                # Usar el nombre de parámetro correcto para esta implementación
                param_nombre = parametros.get(nombre_impl, 'min_support')
                kwargs = {param_nombre: min_support}

                frequent_itemsets = funcion_impl(dataset, **kwargs)
                fin = time.time()

                tiempo_ejecucion = fin - inicio
                print(f"Tiempo: {tiempo_ejecucion:.4f}s, Itemsets encontrados: {len(frequent_itemsets)}")

                # Guardar resultados
                resultados[nombre_impl].append({
                    'dataset': dataset_name,
                    'min_support': min_support,
                    'tiempo': tiempo_ejecucion,
                    'n_itemsets': len(frequent_itemsets)
                })

    return resultados

In [ ]:
%matplotlib inline

In [ ]:
def visualizar_resultados_simple(resultados, datasets, min_supports):

    # Verificar si hay resultados
    if not resultados:
        print("No hay resultados para visualizar.")
        return

    # Mostrar resultados en formato de tabla
    print("\n===== RESULTADOS DE LA COMPARACIÓN =====")

    # Para cada dataset
    for dataset_name in datasets.keys():
        print(f"\n\nDATASET: {dataset_name}")
        print("-" * 80)

        # Crear encabezado
        header = "| Soporte | "
        for impl_name in resultados.keys():
            header += f"{impl_name} (tiempo) | {impl_name} (itemsets) | "
        print(header)
        print("-" * len(header))

        # Mostrar resultados para cada soporte
        for min_support in min_supports:
            row = f"| {min_support:.3f} | "

            for impl_name in resultados.keys():
                # Buscar el resultado para este dataset, implementación y soporte
                result = None
                for r in resultados[impl_name]:
                    if r['dataset'] == dataset_name and r['min_support'] == min_support:
                        result = r
                        break

                if result:
                    row += f"{result['tiempo']:.4f}s | {result['n_itemsets']} | "
                else:
                    row += "N/A | N/A | "

            print(row)

    # Crear gráfico de barras simple
    print("\n\nCreando gráfico de barras...")

    # Configurar el gráfico
    plt.figure(figsize=(12, 6))

    # Para cada dataset, crear un gráfico
    for i, dataset_name in enumerate(datasets.keys()):
        plt.subplot(len(datasets), 1, i+1)

        # Preparar datos
        bar_positions = np.arange(len(min_supports))
        bar_width = 0.35

        # Para cada implementación
        for j, impl_name in enumerate(resultados.keys()):
            # Extraer tiempos
            tiempos = []
            for min_s in min_supports:
                tiempo = 0
                for r in resultados[impl_name]:
                    if r['dataset'] == dataset_name and r['min_support'] == min_s:
                        tiempo = r['tiempo']
                        break
                tiempos.append(tiempo)

            # Graficar barras
            plt.bar(bar_positions + j*bar_width/len(resultados), tiempos,
                   width=bar_width/len(resultados), label=impl_name)

        # Configurar gráfico
        plt.title(f'Comparación - {dataset_name}')
        plt.xlabel('Soporte mínimo')
        plt.ylabel('Tiempo (segundos)')
        plt.xticks(bar_positions, [str(s) for s in min_supports])
        plt.legend()
        plt.grid(axis='y', linestyle='--', alpha=0.7)

    plt.tight_layout()

    # Guardar y mostrar
    ruta_guardado = r'c:\Users\stick\Desktop\4Geeks Academy Projects\Apriori-Latex-Project-Final\Apriori-Latex-Project-Final\comparacion_rendimiento.png'
    try:
        plt.savefig(ruta_guardado)
        print(f"Gráfico guardado en: {ruta_guardado}")
    except Exception as e:
        print(f"Error al guardar el gráfico: {str(e)}")

    # Mostrar el gráfico
    try:
        plt.show()
        print("Gráfico mostrado correctamente.")
    except Exception as e:
        print(f"Error al mostrar el gráfico: {str(e)}")

In [ ]:
def ejecutar_comparacion():

    print("Iniciando comparación de rendimiento...")

    # Definir implementaciones a comparar
    implementaciones = {
        'Apriori Original': apriori_original,
        'Apriori Optimizado': apriori_con_poda_optimizada
    }

    # Generar datasets sintéticos
    print("Generando datasets sintéticos...")
    datasets = {
        'Dataset Pequeño': generar_dataset_sintetico(100, 20, 5),
        'Dataset Mediano': generar_dataset_sintetico(1000, 50, 8)
    }

    # Definir umbrales de soporte a probar
    min_supports = [0.1, 0.05, 0.02]

    # Ejecutar comparación
    print("Ejecutando comparación...")
    resultados = comparar_rendimiento(datasets, min_supports, implementaciones, parametros=None)

    # Verificar si hay resultados
    if not resultados:
        print("La comparación no generó resultados.")
        return None

    # Imprimir resultados en bruto para depuración
    print("\nResultados en bruto:")
    for impl_name, results in resultados.items():
        print(f"\n{impl_name}:")
        for r in results:
            print(f"  {r}")

    # Visualizar resultados
    print("\nVisualizando resultados...")
    visualizar_resultados_simple(resultados, datasets, min_supports)

    return resultados

# Ejecutar la comparación
print("Ejecutando la comparación de rendimiento...")
resultados = ejecutar_comparacion()
print("Comparación completada.")

## Aplicacion Fuerza bruta del algoritmo apriori

In [ ]:
#Aplicamos una funcion de fuerza bruta del algoritmo para compara con apriori
def apriori_fuerza_bruta(transactions, min_support=0.5, max_length=None):

    # Convertimos transacciones a conjuntos si no lo son...
    transactions = [set(transaction) for transaction in transactions]
    n_transactions = len(transactions)
    min_support_count = min_support * n_transactions

    # encontrar todos los items únicos
    all_items = set()
    for transaction in transactions:
        all_items.update(transaction)

    print(f"Número total de items únicos: {len(all_items)}")

    # Almacenamos
    all_frequent_itemsets = {}

    # observamos la longitud máxima si no se especifica (esto revisar info al respecto)
    if max_length is None:
        max_length = len(all_items)

    # generar y evaluar todos los posibles itemsets de tamaño 1 hasta max_length
    for k in range(1, max_length + 1):
        start_time = time.time()

        # Generar todos los posibles k-itemsets (sin poda)
        all_candidates = list(combinations(all_items, k))
        print(f"Generados {len(all_candidates)} candidatos de tamaño {k}")

        # Conteo de soporte por candidato
        itemset_counts = defaultdict(int)
        for candidate in all_candidates:
            candidate_set = frozenset(candidate)
            for transaction in transactions:
                if candidate_set.issubset(transaction):
                    itemset_counts[candidate_set] += 1

        # filtrico
        k_frequent_itemsets = {itemset: count for itemset, count in itemset_counts.items()
                              if count >= min_support_count}


        all_frequent_itemsets.update(k_frequent_itemsets)

        # estadísticas de esta iteración
        end_time = time.time()
        print(f"Iteración {k}: Encontrados {len(k_frequent_itemsets)} itemsets frecuentes en {end_time - start_time:.4f} segundos")

        # Si no hay no habra en la siguiente
        if not k_frequent_itemsets:
            break

    return all_frequent_itemsets

In [ ]:
def apriori_lotes(transactions_list, min_support, batch_size=50000):

    n_total = len(transactions_list)
    print(f"Procesando {n_total} transacciones con soporte mínimo {min_support}")

    # Fase 1: Contar 1-itemsets en lotes
    item_counts = {}
    n_batches = (n_total + batch_size - 1) // batch_size
    print(f"Fase 1: Generando 1-itemsets frecuentes en {n_batches} lotes.")
    for i in range(n_batches):
        start_idx = i * batch_size
        end_idx = min((i + 1) * batch_size, n_total)
        print(f"  Procesando lote {i+1}/{n_batches} (transacciones {start_idx} - {end_idx})")
        batch_transactions = transactions_list[start_idx:end_idx]
        for transaction in batch_transactions:
            for item in transaction:
                item_counts[item] = item_counts.get(item, 0) + 1

    frequent_1_itemsets = []
    support_dict = {}
    for item, count in item_counts.items():
        support = count / n_total
        if support >= min_support:
            frequent_1_itemsets.append([item])
            support_dict[tuple([item])] = support
    print(f"Se encontraron {len(frequent_1_itemsets)} 1-itemsets frecuentes.")

    all_frequent_itemsets = {1: frequent_1_itemsets}
    k = 1
    while all_frequent_itemsets.get(k, []):
        print(f"Fase {k+1}: Generando {k+1}-itemsets frecuentes.")
        if len(all_frequent_itemsets[k]) > 1000:
            print(f"  Demasiados itemsets frecuentes ({len(all_frequent_itemsets[k])}). Limitando a 1000 con mayor soporte.")
            sorted_itemsets = sorted(all_frequent_itemsets[k],
                                     key=lambda x: support_dict.get(tuple(x), 0),
                                     reverse=True)[:1000]
            all_frequent_itemsets[k] = sorted_itemsets

        candidatos = generar_candidatos(all_frequent_itemsets[k], k, support_dict=support_dict)
        if len(candidatos) > 100000:
            print(f"  Demasiados candidatos ({len(candidatos)}). Aumentando temporalmente el umbral de soporte.")
            temp_min_support = min_support * 2
            filtered_itemsets = [itemset for itemset in all_frequent_itemsets[k] if support_dict.get(tuple(itemset), 0) >= temp_min_support]
            candidatos = generar_candidatos(filtered_itemsets, k, support_dict=support_dict)

        candidatos_podados = poda_apriori(candidatos, all_frequent_itemsets[k], k)
        print(f"  Generados {len(candidatos)} candidatos, {len(candidatos_podados)} después de poda.")
        if not candidatos_podados:
            break

        # Contar soporte de candidatos en lotes
        candidato_counts = {tuple(c): 0 for c in candidatos_podados}
        for i in range(n_batches):
            start_idx = i * batch_size
            end_idx = min((i + 1) * batch_size, n_total)
            print(f"  Contando candidatos en lote {i+1}/{n_batches} (transacciones {start_idx} - {end_idx})")
            batch_transactions = transactions_list[start_idx:end_idx]
            for transaction in batch_transactions:
                transaction_set = set(transaction)
                for candidato in candidatos_podados:
                    if all(item in transaction_set for item in candidato):
                        candidato_counts[tuple(candidato)] += 1

        frequent_itemsets_k_plus_1 = []
        for candidato, count in candidato_counts.items():
            support = count / n_total
            if support >= min_support:
                frequent_itemsets_k_plus_1.append(list(candidato))
                support_dict[tuple(sorted(candidato))] = support

        print(f"  Se encontraron {len(frequent_itemsets_k_plus_1)} {k+1}-itemsets frecuentes.")
        if frequent_itemsets_k_plus_1:
            all_frequent_itemsets[k+1] = frequent_itemsets_k_plus_1
            k += 1
        else:
            break
    return all_frequent_itemsets, support_dict


In [ ]:
def generar_reglas(itemsets_frecuentes, support_dict, min_confidence=0.5):

    rules = []
    for k in range(2, len(itemsets_frecuentes) + 1):
        if k not in itemsets_frecuentes:
            continue
        for itemset in itemsets_frecuentes[k]:
            itemset_support = support_dict.get(tuple(sorted(itemset)), 0)
            for i in range(1, len(itemset)):
                for antecedente_items in combinations(itemset, i):
                    antecedente = list(antecedente_items)
                    consecuente = [item for item in itemset if item not in antecedente]
                    if not antecedente or not consecuente:
                        continue
                    antecedente_support = support_dict.get(tuple(sorted(antecedente)), 0)
                    if antecedente_support == 0:
                        continue
                    confidence = itemset_support / antecedente_support
                    if confidence >= min_confidence:
                        consecuente_support = support_dict.get(tuple(sorted(consecuente)), 0)
                        if consecuente_support == 0:
                            continue
                        lift = confidence / consecuente_support
                        rules.append({
                            'antecedent': antecedente,
                            'consequent': consecuente,
                            'support': itemset_support,
                            'confidence': confidence,
                            'lift': lift
                        })
    if rules:
        rulesDf = pd.DataFrame(rules)
        rulesDf = rulesDf.sort_values('lift', ascending=False).reset_index(drop=True)
        return rulesDf
    else:
        return pd.DataFrame(columns=['antecedent', 'consequent', 'support', 'confidence', 'lift'])


In [ ]:

# Ejecutar el algoritmo Apriori manual
min_support = 0.01  # Ajusta según tu dataset
start_time = time.time()

# Aquí usamos la lista de transacciones obtenida en la SECCIÓN 1
transactions_list = transactions  # Asegúrate de haber generado `transactions` previamente
all_frequent_itemsets, support_dict = apriori_lotes(transactions_list, min_support)

manual_time = time.time() - start_time
print(f"\nTiempo del Apriori manual: {manual_time:.2f} segundos")

# Generar reglas a partir de los itemsets frecuentes
rules_manual = generar_reglas(all_frequent_itemsets, support_dict, min_confidence=0.3)
print("\nReglas generadas manualmente (primeras 5):")
print(rules_manual.head())

## Documentacion de la complejidad computacional

In [ ]:
def documentar_complejidad_experimental():

    print("=== ANÁLISIS DE COMPLEJIDAD COMPUTACIONAL DEL ALGORITMO APRIORI ===")
    print("Este análisis experimental valida las afirmaciones teóricas presentadas en el documento.")

    # 1. Generación de 1-itemsets
    print("\n1. Generación de 1-itemsets:")
    print("   Complejidad Teórica: O(n × m) donde n = número de transacciones, m = número de items únicos")

    print("\n   Mediciones experimentales (tiempo en segundos):")
    print("   | Transacciones (n) | Items (m) | Tiempo | Ratio n×m/tiempo |")
    print("   |------------------|-----------|--------|------------------|")

    # Variar n manteniendo m constante
    m_constante = 50
    for n in [100, 500, 1000, 5000]:
        dataset = generar_dataset_sintetico(n, m_constante, m_constante//4)

        start_time = time.time()
        item_counts = contar_items_individuales(dataset)
        end_time = time.time()

        tiempo = end_time - start_time
        ratio = (n * m_constante) / (tiempo * 1000)  # Normalizado para mejor visualización

        print(f"   | {n:16d} | {m_constante:9d} | {tiempo:.6f} | {ratio:.2f} |")

    # Variar m manteniendo n constante
    n_constante = 1000
    for m in [20, 50, 100, 200]:
        dataset = generar_dataset_sintetico(n_constante, m, m//4)

        start_time = time.time()
        item_counts = contar_items_individuales(dataset)
        end_time = time.time()

        tiempo = end_time - start_time
        ratio = (n_constante * m) / (tiempo * 1000)  # Normalizado

        print(f"   | {n_constante:16d} | {m:9d} | {tiempo:.6f} | {ratio:.2f} |")

    # 2. Generación de Candidatos
    print("\n2. Generación de Candidatos (k+1 a partir de k-itemsets):")
    print("   Complejidad Teórica: O(l² × k) donde l = número de k-itemsets frecuentes, k = tamaño del itemset")

    print("\n   Mediciones experimentales (tiempo en segundos):")
    print("   | Itemsets (l) | Tamaño (k) | Tiempo | Ratio l²×k/tiempo |")
    print("   |-------------|------------|--------|-------------------|")

    # Variar l manteniendo k constante
    k_constante = 3
    for l in [10, 50, 100, 200]:
        # Crear itemsets frecuentes sintéticos
        itemsets = [frozenset([f'item_{i}' for i in range(j, j+k_constante)])
                   for j in range(l)]

        start_time = time.time()
        candidatos = generar_candidatos(itemsets, k_constante)
        end_time = time.time()

        tiempo = end_time - start_time
        ratio = (l**2 * k_constante) / (tiempo * 1000)  # Normalizado

        print(f"   | {l:11d} | {k_constante:10d} | {tiempo:.6f} | {ratio:.2f} |")

    # Variar k manteniendo l constante
    l_constante = 100
    for k in [2, 3, 4, 5]:
        # Crear itemsets frecuentes sintéticos
        itemsets = [frozenset([f'item_{i}' for i in range(j, j+k)])
                   for j in range(l_constante)]

        start_time = time.time()
        candidatos = generar_candidatos(itemsets, k)
        end_time = time.time()

        tiempo = end_time - start_time
        ratio = (l_constante**2 * k) / (tiempo * 1000)  # Normalizado

        print(f"   | {l_constante:11d} | {k:10d} | {tiempo:.6f} | {ratio:.2f} |")

    # 3. Poda Apriori
    print("\n3. Poda Apriori:")
    print("   Complejidad Teórica sin optimización: O(c × k × l)")
    print("   Complejidad Teórica con optimización: O(c × k)")

    print("\n   Mediciones experimentales (tiempo en segundos):")
    print("   | Candidatos (c) | Tamaño (k) | Itemsets (l) | Sin optimización | Con optimización | Mejora (%) |")
    print("   |---------------|------------|--------------|------------------|------------------|------------|")

    for c in [20, 50, 100]:
        for k in [3, 4]:
            l = c // 2  # Relación arbitraria entre c y l para el experimento

            # Crear datos sintéticos
            candidatos = [sorted([f'item_{i}' for i in range(j, j+k)])
                         for j in range(c)]
            itemsets_frecuentes = [tuple(sorted([f'item_{i}' for i in range(j, j+k-1)]))
                                  for j in range(l)]

            # Medir tiempo sin optimización
            start_time = time.time()
            poda_apriori(candidatos, itemsets_frecuentes, k)
            end_time = time.time()
            tiempo_sin_opt = end_time - start_time

            # Medir tiempo con optimización
            start_time = time.time()
            apriori_con_poda_optimizada(candidatos, itemsets_frecuentes, k)
            end_time = time.time()
            tiempo_con_opt = end_time - start_time

            # Calcular mejora
            if tiempo_sin_opt > 0:
                mejora = (1 - tiempo_con_opt / tiempo_sin_opt) * 100
            else:
                mejora = 0

            print(f"   | {c:13d} | {k:10d} | {l:12d} | {tiempo_sin_opt:.6f} | {tiempo_con_opt:.6f} | {mejora:.2f} |")

    # 4. Cálculo de Soporte
    print("\n4. Cálculo de Soporte para Candidatos:")
    print("   Complejidad Teórica: O(n × c × k)")

    print("\n   Mediciones experimentales (tiempo en segundos):")
    print("   | Transacciones (n) | Candidatos (c) | Tamaño (k) | Tiempo | Ratio n×c×k/tiempo |")
    print("   |-------------------|---------------|------------|--------|--------------------|")

    for n in [100, 500, 1000]:
        for c in [10, 50]:
            for k in [2, 3, 4]:
                # Generar datos sintéticos
                dataset = generar_dataset_sintetico(n, 100, 20)
                candidatos = [frozenset([f'item_{i}' for i in range(j, j+k)])
                             for j in range(c)]

                # Medir tiempo
                start_time = time.time()
                count_support_optimized(dataset, candidatos)
                end_time = time.time()

                tiempo = end_time - start_time
                ratio = (n * c * k) / (tiempo * 1000)  # Normalizado

                print(f"   | {n:17d} | {c:13d} | {k:10d} | {tiempo:.6f} | {ratio:.2f} |")

    # 5. Complejidad Total y Factores que Afectan el Rendimiento
    print("\n=== COMPLEJIDAD TOTAL Y FACTORES QUE AFECTAN EL RENDIMIENTO ===")

    # 5.1 Efecto del umbral de soporte mínimo
    print("\n5.1 Efecto del Umbral de Soporte Mínimo:")
    print("   | Soporte Mínimo | Tiempo (s) | Itemsets Encontrados |")
    print("   |----------------|------------|----------------------|")

    dataset_fijo = generar_dataset_sintetico(1000, 50, 15)
    for min_support in [0.1, 0.05, 0.02, 0.01]:
        start_time = time.time()
        frequent_itemsets = apriori_original(dataset_fijo, min_support=min_support)
        end_time = time.time()

        tiempo = end_time - start_time

        print(f"   | {min_support:.4f} | {tiempo:.6f} | {len(frequent_itemsets):20d} |")

    # 5.2 Efecto del número de items únicos
    print("\n5.2 Efecto del Número de Items Únicos:")
    print("   | Items Únicos | Tiempo (s) | Itemsets Encontrados |")
    print("   |-------------|------------|----------------------|")

    n_fijo = 1000
    min_support_fijo = 0.05
    for m in [20, 50, 100, 200]:
        dataset = generar_dataset_sintetico(n_fijo, m, m//3)

        start_time = time.time()
        frequent_itemsets = apriori_original(dataset, min_support=min_support_fijo)
        end_time = time.time()

        tiempo = end_time - start_time

        print(f"   | {m:11d} | {tiempo:.6f} | {len(frequent_itemsets):20d} |")

    # 5.3 Efecto de la longitud de las transacciones
    print("\n5.3 Efecto de la Longitud de las Transacciones:")
    print("   | Longitud Media | Tiempo (s) | Itemsets Encontrados |")
    print("   |---------------|------------|----------------------|")

    n_fijo = 1000
    m_fijo = 100
    min_support_fijo = 0.05
    for longitud in [10, 20, 30, 40]:
        dataset = generar_dataset_sintetico(n_fijo, m_fijo, longitud)

        start_time = time.time()
        frequent_itemsets = apriori_original(dataset, min_support=min_support_fijo)
        end_time = time.time()

        tiempo = end_time - start_time

        print(f"   | {longitud:13d} | {tiempo:.6f} | {len(frequent_itemsets):20d} |")

    # Visualización gráfica de los resultados
    print("\n=== VISUALIZACIÓN GRÁFICA DE LOS RESULTADOS ===")
    print("Generando gráficos para visualizar los resultados experimentales...")

In [ ]:
def visualizar_resultados_experimentales():
    """Genera visualizaciones gráficas de los resultados experimentales"""
    # Datos de ejemplo para las gráficas (estos deberían ser reemplazados por datos reales)
    # 1. Efecto del soporte mínimo
    soportes = [0.1, 0.05, 0.02, 0.01]
    tiempos = [0.5, 1.2, 3.5, 8.7]
    itemsets = [10, 25, 80, 210]

    plt.figure(figsize=(12, 8))

    # Gráfico 1: Tiempo vs Soporte mínimo
    plt.subplot(2, 2, 1)
    plt.plot(soportes, tiempos, 'o-', color='blue')
    plt.title('Tiempo de ejecución vs Soporte mínimo')
    plt.xlabel('Soporte mínimo')
    plt.ylabel('Tiempo (s)')
    plt.grid(True, linestyle='--', alpha=0.7)

    # Gráfico 2: Itemsets encontrados vs Soporte mínimo
    plt.subplot(2, 2, 2)
    plt.plot(soportes, itemsets, 'o-', color='green')
    plt.title('Itemsets frecuentes vs Soporte mínimo')
    plt.xlabel('Soporte mínimo')
    plt.ylabel('Número de itemsets')
    plt.grid(True, linestyle='--', alpha=0.7)

    # Gráfico 3: Complejidad de generación de candidatos
    l_values = [10, 50, 100, 200]
    tiempos_gen = [0.001, 0.02, 0.08, 0.3]

    plt.subplot(2, 2, 3)
    plt.plot(l_values, tiempos_gen, 'o-', color='red')
    plt.title('Tiempo de generación de candidatos vs Tamaño de L')
    plt.xlabel('Número de itemsets frecuentes (l)')
    plt.ylabel('Tiempo (s)')
    plt.grid(True, linestyle='--', alpha=0.7)

    # Gráfico 4: Comparación de poda con y sin optimización
    candidatos = [20, 50, 100]
    tiempos_sin_opt = [0.01, 0.05, 0.2]
    tiempos_con_opt = [0.002, 0.01, 0.04]

    plt.subplot(2, 2, 4)
    x = np.arange(len(candidatos))
    width = 0.35

    plt.bar(x - width/2, tiempos_sin_opt, width, label='Sin optimización')
    plt.bar(x + width/2, tiempos_con_opt, width, label='Con optimización')

    plt.title('Comparación de métodos de poda')
    plt.xlabel('Número de candidatos')
    plt.xticks(x, candidatos)
    plt.ylabel('Tiempo (s)')
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.7)

    plt.tight_layout()
    plt.savefig(r'c:\Users\stick\Desktop\4Geeks Academy Projects\Apriori-Latex-Project-Final\Apriori-Latex-Project-Final\complejidad_experimental.png')
    plt.show()

In [ ]:
try:
    documentar_complejidad_experimental()
    visualizar_resultados_experimentales()
except Exception as e:
    print(f"Error al ejecutar la documentación experimental: {str(e)}")
    import traceback
    traceback.print_exc()